In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import sparse
from scipy import optimize
from scipy import signal
from functools import reduce
from collections import defaultdict
import pprint
from tqdm.notebook import tqdm
import pandas as pd # Only here to print arrays nicely

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# TOC
1. [1x1 Beam Functions](#1x1)
2. [NxN Beam Complications](#NxN)
3. [Playground](#playground)

### Simplified case for 1x1 beam <a name="1x1"></a>

In [3]:
def get_num_baselines(Nside):
    # For square of N telescopes, should be 2N-2*sqrt(N) + 1
    # Fix to a corner and there are N-1 options for unique baselines. 
    # Flip over to other side (over x or y axis) and get another N-1 options
    # Duplicated are the pure x-axis and pure y-axis so -2*(sqrt(N)-1)
    # Final +1 is from 0 baseline
    
    N_bases = 2*Nside**2 - 2*Nside
    return int(N_bases)

rand_phases = lambda x: np.random.uniform(0, 2*np.pi, x)
zero_weight = lambda x, d: x/d if d else 0

def make_gains(Nside):
    # Create complex gains with either (amplitude, phase) or (real, imaginary)
    Nant = Nside**2
    gain_amp = np.random.normal(1, .05, Nant)
    gain_phase = rand_phases(Nant)
    tgain = gain_amp*np.exp(1j*gain_phase)    
    return tgain

def make_data(Nside, gains, noise=0.1):
    Nant = Nside**2
    Nbase = get_num_baselines(Nside)
    vis_true = np.random.normal(0,1,size=(Nbase,2)).view(np.complex128).flatten() ## size of unique baselines
    ant_i, ant_j, visndx, data = [], [], [], []
    ndx=0
    ndx2base={}
    base2ndx={}
    for i in range(Nant):
        xi,yi=np.unravel_index(i,(Nside,Nside))
        for j in range (i+1,Nant):
            xj,yj=np.unravel_index(j,(Nside,Nside))
            assert (xj>=xi)
            baseline = (xj-xi,yj-yi)
            if baseline in base2ndx:
                cndx = base2ndx[baseline]
            else:
                cndx = ndx
                base2ndx[baseline]=ndx
                ndx2base[ndx]=baseline
                ndx+=1
            ant_i.append(i)
            ant_j.append(j)
            visndx.append(cndx)
            data.append(vis_true[cndx]*gains[i]*np.conj(gains[j]))
            
    assert(ndx==Nbase)
    ant_i = np.array(ant_i)
    ant_j = np.array(ant_j)
    visndx = np.array(visndx)
    data = np.array(data)
    noise = np.random.normal(0,noise,size=(len(data),2)).view(np.complex128).flatten() ## size of unique baselines
    data += noise
    return vis_true, data, ant_i, ant_j, visndx, ndx2base, base2ndx

In [4]:
def make_pred(gains, vis, ant_i, ant_j, visndx):
    gains_i = gains[ant_i]
    cgains_j = np.conj(gains[ant_j])
    pred = gains_i*cgains_j*vis[visndx]
    return pred

In [5]:
def chi2(data, gains, vis, ant_i, ant_j, visndx, noise=0.1):
    pred = make_pred(gains, vis, ant_i, ant_j, visndx)
    chi2 = np.abs((data - pred)**2).sum()/(noise**2)
    dof = len(data)*2
    return chi2, dof

### NxN beam complications <a name="NxN"></a>

In [6]:
def get_weighted_array(alpha, Nspacing, numdraws=1e5):
    Nbeam = Nspacing**2
    rmax = alpha*.5
    ndraws = int(numdraws)
    spacing = np.linspace(0,1,Nspacing+1)
    centered_spacing = spacing - .5
    empty_weight_beam = np.zeros((Nspacing, Nspacing))
    
    for i in range(Nbeam):
        xi, yi = np.unravel_index(i, (Nspacing, Nspacing))
        draws = np.array([np.random.uniform(centered_spacing[xi], centered_spacing[xi+1], ndraws), np.random.uniform(centered_spacing[yi], centered_spacing[yi+1], ndraws)])
        dist = np.linalg.norm(draws, axis=0)
        empty_weight_beam[xi, yi] = np.sum(dist < rmax)/ndraws
    sym_beam = .5*(empty_weight_beam + empty_weight_beam.T)
    return sym_beam

In [319]:
def n_mesh(br):
    nmesh = lambda x: tuple(np.meshgrid(range(x[0]-br, x[0]+br+1), range(x[1]-br, x[1]+br+1), indexing='ij'))
    return nmesh

five_mesh = n_mesh(2)

In [320]:
def make_uv_grid(Nside):
    uv_size = Nside*2 - 1
    center = (Nside-1,Nside-1)
    npcenter = np.array(center)
    img_size = (uv_size, uv_size)
    random_image = np.random.normal(0, 1, img_size)
    centered_uv = np.fft.fftshift(np.fft.fft2(random_image))
#     centered_uv[center] = 0
    topleft_uv = np.fft.ifftshift(centered_uv)
    
    return centered_uv, topleft_uv, npcenter

In [321]:
def make_visibilities(Nside, beam_radius):
    n = Nside
    br = beam_radius
    
    # Useful shapes/sizes to have on hand
    orig_shape = (2*n-1, n)
    new_shape = tuple([i*br + (br - 1) for i in orig_shape])
    new_size = np.prod(new_shape)
    center = (int((new_shape[0]-1)/2), br-1)
    
    #Matrices of visibiltiies and possible indices
    random_vis = np.random.normal(0, 1, (new_shape[0], new_shape[1], 2)).view(np.complex128)
    poss_index = np.arange(new_size).reshape(new_shape)
    
    #Clear off the leftmost columns to remove redundant conjugate stuff
    poss_index[0:center[0], 0:br] = 0
    poss_index[center[0]:, 0:br-1] = 0
    
    oversampled_baselines = poss_index.nonzero()[0].shape[0]
    visib = np.zeros(oversampled_baselines, dtype=np.complex128)
    #Baseline - index dictionaries
    new_ndx = 0
    n2b = {}
    b2n = {}
    n2true = {}
    true2n = {}
    for i in range(new_size):
        xi,yi = np.unravel_index(i, new_shape)
        if poss_index[xi,yi] == 0:
            continue
        else:
            baseline = (xi-center[0], yi-center[1])
            if baseline[0] < 0:
                baseline = (-1*baseline[0], -1*baseline[1])

            if baseline in b2n:
                cndx = new_ndx
            else:
                cndx = new_ndx
                b2n[baseline] = new_ndx
                n2b[new_ndx] = baseline
                if tuple(np.mod(baseline, br)) == (0,0):
                    modtuple = tuple(np.floor_divide(baseline, 3))
                    true2n[modtuple] = new_ndx
                    n2true[new_ndx] = modtuple
                new_ndx += 1
        visib[cndx] = random_vis[xi,yi]
    return n2b, b2n, n2true, true2n, visib

In [352]:
def vis_to_grid(visib, ndx2base, size):
    # Double the size to make sure to avoid rewriting values with negative indexing
    uv_size = (2*size[0], 2*size[1])
    new_grid = np.zeros(uv_size, dtype=np.complex128)
    for i,v in enumerate(visib):
        base = ndx2base[i]
        inv_base = tuple(-1*np.array(base))
        new_grid[base] = v
        new_grid[inv_base] = np.conj(v)
    return new_grid

In [353]:
def make_data_grid(Nside, gains, conv_beam, noise=0.1, verbose=True):
    Nant = Nside**2
    Nbase = get_num_baselines(Nside)
    beam_radius = int((conv_beam.shape[0]-1)/2) + 1
    mesh = n_mesh(beam_radius-1)
    
    n2b, b2n, n2t, t2n, visib = make_visibilities(Nside, beam_radius)
    if verbose:
        print("Made visib")
    orig_shape = (2*Nside-1, Nside)
    new_shape = tuple([i*beam_radius + (beam_radius - 1) for i in orig_shape])
    
    vis_grid = vis_to_grid(visib, n2b, new_shape)
    if verbose:
        print("Made grid")
    
    ant_i, ant_j, visndx, data, datandx = [], [], [], [], []
    
    for i in range(Nant):
        xi,yi=np.unravel_index(i,(Nside, Nside))
        for j in range (i+1,Nant):
            xj,yj=np.unravel_index(j,(Nside, Nside))
            assert (xj>=xi)
            baseline = (xj-xi,yj-yi)
            
            virtual_n = t2n[baseline]
            grid_base = n2b[virtual_n]
            
            virtual_points = mesh(grid_base)
            data_sum = (vis_grid[virtual_points].T * gains[i] * np.conj(gains[j]) * conv_beam).sum()
            
            ant_i.append(i)
            ant_j.append(j)
            visndx.append(virtual_n)
            data.append(data_sum)
            datandx.append(virtual_points)
    
    if verbose:
        print("Created data")
    
    ant_i = np.array(ant_i)
    ant_j = np.array(ant_j)
    visndx = np.array(visndx)
    data = np.array(data)
    noise = np.random.normal(0,noise,size=(len(data),2)).view(np.complex128).flatten() ## size of unique baselines
    data += noise
    return visib, data, ant_i, ant_j, visndx, datandx, n2b, b2n, n2t, t2n, vis_grid

In [354]:
def vector_b2n(b2n, datandx):
    newndx = []
    for l in datandx:
        x,y = l
        flattened = []
        ndx_size = x.size
        ndx_shape = x.shape
        for i in range(ndx_size):
            point = np.unravel_index(i, ndx_shape)
            k = np.array([x[point], y[point]])
            key = tuple(k)
            if key in b2n:
                flattened.append(b2n[key])
            else:
                key = tuple(-1*k)
                flattened.append(-1*b2n[key])
        newndx.append(flattened)
    newndx = np.array(newndx)
    return newndx

In [355]:
def conjugate_visib(vis, ndxs):
    flat = []
    for i in ndxs:
        if i >= 0:
            flat.append(vis[i])
        else:
            flat.append(np.conj(vis[-1*i]))
    return np.array(flat)

In [356]:
def forward_model(vis, gains, conv_beam, ant_i, ant_j, n2b, datandx):
    Nside = int(np.sqrt(len(gains_true)))
    beam_radius = int((conv_beam.shape[0]-1)/2) + 1
    
    orig_shape = (2*Nside-1, Nside)
    new_shape = tuple([i*beam_radius + (beam_radius - 1) for i in orig_shape])
    vis_grid = vis_to_grid(vis, n2b, new_shape)
    
    pred = []
    
    for i in range(len(ant_i)):
        pred.append((vis_grid[datandx[i]] * gains[ant_i[i]] * np.conj(gains[ant_j[i]]) * conv_beam).sum())
    pred = np.array(pred)
    return pred

In [647]:
def flat_model(vis, gains, conv_beam, ant_i, ant_j, flatndx):
    flatbeam = conv_beam.flatten()
    gains_i = gains[ant_i]
    gains_j = np.conj(gains[ant_j])
    flatbread = np.array([conjugate_visib(vis, i)*flatbeam for i in flatndx])
    pred = np.sum(flatbread, axis=1)*gains_i*gains_j
    return pred

In [652]:
def gen_chi2(data, pred, noise=0.1):
    chi2 = np.abs((data - pred)**2).sum()/(noise**2)
    dof = len(data)*2
    return chi2, dof

In [847]:
Nside = 15
Nant = Nside * Nside
Nbase = get_num_baselines(Nside)
gains_true = make_gains(Nside)
# gains_true = np.ones(Nant, dtype=np.complex128)
# vis_true, data, ant_i, ant_j, visndx, ndx2base, base2ndx = make_data(Nside, gains_true)
weighted_beam = get_weighted_array(2, 3)
conv_beam = signal.convolve(weighted_beam, weighted_beam)
vis_true, data, ant_i, ant_j, visndx, datandx, n2b, b2n, n2t, t2n, tl_grid = make_data_grid(Nside, gains_true, conv_beam, noise=0.1)
flatndx = vector_b2n(b2n, datandx)
data_len = len(data)
no_noise = make_pred(gains_true, vis_true, ant_i, ant_j, visndx)

Made visib
Made grid
Created data


In [848]:
# phased_gains = gains_true*np.exp(1j*rand_phases(Nant)/5)

In [849]:
test_b = forward_model(vis_true, gains_true, conv_beam, ant_i, ant_j, n2b, datandx)
flatbread = flat_model(vis_true, gains_true, conv_beam, ant_i, ant_j, flatndx)

In [850]:
print("Grid solution ", gen_chi2(data, test_b))
print("Flat solution ", gen_chi2(data, flatbread))

Grid solution  (50060.37129582962, 50400)
Flat solution  (50060.37129582963, 50400)


### Below this is my random testing/playground <a name="playground"></a>

In [538]:
real_part = data.real
imag_part = data.imag
big_ans = np.concatenate((real_part, imag_part))

In [539]:
data[0]

(8.572082173967576-12.945855975832021j)

In [540]:
v_size = len(set(np.abs(flatndx).flatten())) + 1

In [541]:
bigA = np.zeros((2*data_len, 2*v_size))

In [542]:
for i,v in enumerate(flatndx):
    absv = np.abs(v)
    bigA[i][absv] = conv_beam.flatten()
    bigA[i+data_len][v_size+absv] = conv_beam.flatten()*np.sign(v)

In [543]:
bigA.shape

(50400, 7922)

In [544]:
shitsol = sparse.linalg.lsqr(bigA, big_ans)[0]

In [545]:
comb_sol = shitsol[0:v_size] + 1j*shitsol[v_size:]

In [547]:
comb_sol - vis_true

array([ 0.18113216+0.57474405j,  1.09030433+0.89907105j,
       -1.26751616+0.50466849j, ..., -0.41401121-1.0902055j ,
       -0.32645529-0.26837623j,  0.74284627+0.95022292j])

In [1004]:
# bad_guess = np.random.normal(0,1,(v_size,2)).view(dtype=np.complex128).flatten()
bad_guess = vis_true.copy()
bad_sol = flat_model(bad_guess, gains_true, conv_beam, ant_i, ant_j, flatndx)
gen_chi2(data, bad_sol)

(50060.37129582963, 50400)

In [1005]:
postage = conv_beam.flatten()

In [1006]:
worthy = np.array(list(n2t.keys()))

In [1016]:
(data[vis_filter][0]*np.conj(postage_gain)[0])/denom[0]

array([-6.54911643+42.7901869j , -3.27455821+21.39509345j,
       -2.18303881+14.26339563j, -3.27455821+21.39509345j,
       -6.54911643+42.7901869j , -3.27455821+21.39509345j,
       -1.63727911+10.69754673j, -1.0915194  +7.13169782j,
       -1.63727911+10.69754673j, -3.27455821+21.39509345j,
       -2.18303881+14.26339563j, -1.0915194  +7.13169782j,
       -0.7276796  +4.75446521j, -1.0915194  +7.13169782j,
       -2.18303881+14.26339563j, -3.27455821+21.39509345j,
       -1.63727911+10.69754673j, -1.0915194  +7.13169782j,
       -1.63727911+10.69754673j, -3.27455821+21.39509345j,
       -6.54911643+42.7901869j , -3.27455821+21.39509345j,
       -2.18303881+14.26339563j, -3.27455821+21.39509345j,
       -6.54911643+42.7901869j ])

In [1019]:
vis_true[flatndx[vis_filter][0]]

array([-0.68475733+2.48596873j, -1.21277244-1.76633059j,
        0.38811293+1.48790256j,  0.69475538+0.07013047j,
        1.13519156-0.32153291j,  0.7379021 +1.05524192j,
        1.2058483 +1.31474819j,  0.48744833-0.9765338j ,
        1.11022111+1.13296432j, -1.13233573+0.36075726j,
       -1.01774266+0.42070244j,  0.22484705+0.32629892j,
       -1.72676809+1.4588596j , -0.13492856+1.4947835j ,
        0.19236687-0.61333918j, -2.03853377+1.70156156j,
        1.67878163-0.3532522j ,  1.31163535+0.53540815j,
       -0.8460338 -0.055717j  ,  0.14515933+0.71607523j,
        0.37553013+0.70514269j, -1.12685984+1.00357374j,
       -0.99206222+1.06052488j, -2.34878691-1.82847571j,
        1.09374805+0.7505384j ])

In [1007]:
for w in worthy[:10]:
    if w==zerondx:
        continue
    vis_filter = (visndx==w)
    
    gain_factor = gains_true[ant_i[vis_filter]]*np.conj(gains_true[ant_j[vis_filter]])
    postage_gain = (gain_factor[:,None]*postage[None,:])
    numer = data[vis_filter][:,None]*np.conjugate(postage_gain)
    denom = np.abs(postage_gain)**2
    
    delta_v = numer.sum(axis=0)/denom.sum(axis=0)
    bad_guess[flatndx[vis_filter][0]] += np.conj(delta_v)

In [941]:
for w in worthy:
    if w==zerondx:
        continue
    vis_filter = (visndx==w)
    
    gain_factor = gains_true[ant_i[vis_filter]]*np.conj(gains_true[ant_j[vis_filter]])
    postage_gain = (gain_factor[:,None]*postage[None,:])
    
    delta_v = np.mean(data[vis_filter][:,None]/postage_gain, axis=0)
    bad_guess[flatndx[vis_filter][0]] += delta_v

In [1008]:
bad_sol = flat_model(bad_guess, gains_true, conv_beam, ant_i, ant_j, flatndx)
gen_chi2(data, bad_sol)

(5125784613.202412, 50400)

In [989]:
np.abs(gain_factor)

array([1.00566882, 0.93609095, 0.9419405 , 0.97761385, 1.06171111])

In [825]:
3843634763 > 3870766810

False

In [758]:
gain_factor = gains_true[ant_i[vis_filter]]*np.conj(gains_true[ant_j[vis_filter]])

In [766]:
(gain_factor[:,None]*postage[None,:])

array([[ 0.23237224-1.0239595j ,  0.46474447-2.04791899j,
         0.69711671-3.07187849j,  0.46474447-2.04791899j,
         0.23237224-1.0239595j ,  0.46474447-2.04791899j,
         0.92948894-4.09583798j,  1.39423341-6.14375698j,
         0.92948894-4.09583798j,  0.46474447-2.04791899j,
         0.69711671-3.07187849j,  1.39423341-6.14375698j,
         2.09135012-9.21563547j,  1.39423341-6.14375698j,
         0.69711671-3.07187849j,  0.46474447-2.04791899j,
         0.92948894-4.09583798j,  1.39423341-6.14375698j,
         0.92948894-4.09583798j,  0.46474447-2.04791899j,
         0.23237224-1.0239595j ,  0.46474447-2.04791899j,
         0.69711671-3.07187849j,  0.46474447-2.04791899j,
         0.23237224-1.0239595j ],
       [-0.25582099-1.04530926j, -0.51164199-2.09061853j,
        -0.76746298-3.13592779j, -0.51164199-2.09061853j,
        -0.25582099-1.04530926j, -0.51164199-2.09061853j,
        -1.02328397-4.18123706j, -1.53492596-6.27185558j,
        -1.02328397-4.18123706j, -0.51

In [742]:
ant_j[visndx==768]

array([135, 136, 137, 138, 139, 140, 141, 142, 150, 151, 152, 153, 154,
       155, 156, 157, 165, 166, 167, 168, 169, 170, 171, 172, 180, 181,
       182, 183, 184, 185, 186, 187, 195, 196, 197, 198, 199, 200, 201,
       202, 210, 211, 212, 213, 214, 215, 216, 217])

In [741]:
ant_i[visndx==768]

array([ 7,  8,  9, 10, 11, 12, 13, 14, 22, 23, 24, 25, 26, 27, 28, 29, 37,
       38, 39, 40, 41, 42, 43, 44, 52, 53, 54, 55, 56, 57, 58, 59, 67, 68,
       69, 70, 71, 72, 73, 74, 82, 83, 84, 85, 86, 87, 88, 89])

In [739]:
bigV

array([-7.01401852e-01+0.12855667j,  1.37751191e+00+1.21423085j,
       -6.76871922e-01+1.63387525j,  6.03894778e-01-1.14250081j,
        7.09671917e-02-1.51288659j, -4.85959352e-01+0.01834192j,
        8.64883651e-01-1.52771531j,  2.66959149e-01+1.63910829j,
       -1.05098781e-01+1.72608095j,  1.07551708e+00-0.47287593j,
       -6.46178666e-01+0.87097628j, -6.57957192e-01-0.76530253j,
        1.67089073e+00+1.02808064j, -5.30878356e-02+1.11174005j,
       -1.03937779e-01-0.02624642j, -1.06312863e+00+1.70952402j,
       -1.20832531e-02+1.04804652j,  5.57699315e-01-1.07683029j,
        1.61528370e-01+1.70459352j, -1.14291713e-01+0.04836087j,
       -6.45276530e-01-0.56959394j, -6.81217283e-01+0.22483326j,
        3.91895657e-01+0.30665726j,  1.08178598e+00+0.20186523j,
       -5.58863799e-04-0.2952243j ])